# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fa80076a310>
├── 'a' --> tensor([[-1.5325, -0.6184, -0.0688],
│                   [ 0.6734, -0.1891,  1.4454]])
└── 'x' --> <FastTreeValue 0x7fa80076a9d0>
    └── 'c' --> tensor([[-0.9273, -0.2462, -0.0223,  0.5596],
                        [-2.1601, -1.2822, -1.1878, -1.8289],
                        [-0.3527,  0.2015,  0.6889,  0.7537]])

In [4]:
t.a

tensor([[-1.5325, -0.6184, -0.0688],
        [ 0.6734, -0.1891,  1.4454]])

In [5]:
%timeit t.a

65.5 ns ± 0.0612 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fa80076a310>
├── 'a' --> tensor([[-0.6534,  0.3971, -0.1453],
│                   [-0.9380, -2.2451, -0.4365]])
└── 'x' --> <FastTreeValue 0x7fa80076a9d0>
    └── 'c' --> tensor([[-0.9273, -0.2462, -0.0223,  0.5596],
                        [-2.1601, -1.2822, -1.1878, -1.8289],
                        [-0.3527,  0.2015,  0.6889,  0.7537]])

In [7]:
%timeit t.a = new_value

67.8 ns ± 0.126 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.5325, -0.6184, -0.0688],
               [ 0.6734, -0.1891,  1.4454]]),
    x: Batch(
           c: tensor([[-0.9273, -0.2462, -0.0223,  0.5596],
                      [-2.1601, -1.2822, -1.1878, -1.8289],
                      [-0.3527,  0.2015,  0.6889,  0.7537]]),
       ),
)

In [10]:
b.a

tensor([[-1.5325, -0.6184, -0.0688],
        [ 0.6734, -0.1891,  1.4454]])

In [11]:
%timeit b.a

59.4 ns ± 0.118 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.8661, -0.4200,  0.7819],
               [ 0.4289, -0.1662,  1.1953]]),
    x: Batch(
           c: tensor([[-0.9273, -0.2462, -0.0223,  0.5596],
                      [-2.1601, -1.2822, -1.1878, -1.8289],
                      [-0.3527,  0.2015,  0.6889,  0.7537]]),
       ),
)

In [13]:
%timeit b.a = new_value

490 ns ± 0.118 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

807 ns ± 0.433 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.9 µs ± 19.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

143 µs ± 670 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

143 µs ± 1.03 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fa7d4bd8d30>
├── 'a' --> tensor([[[-1.5325, -0.6184, -0.0688],
│                    [ 0.6734, -0.1891,  1.4454]],
│           
│                   [[-1.5325, -0.6184, -0.0688],
│                    [ 0.6734, -0.1891,  1.4454]],
│           
│                   [[-1.5325, -0.6184, -0.0688],
│                    [ 0.6734, -0.1891,  1.4454]],
│           
│                   [[-1.5325, -0.6184, -0.0688],
│                    [ 0.6734, -0.1891,  1.4454]],
│           
│                   [[-1.5325, -0.6184, -0.0688],
│                    [ 0.6734, -0.1891,  1.4454]],
│           
│                   [[-1.5325, -0.6184, -0.0688],
│                    [ 0.6734, -0.1891,  1.4454]],
│           
│                   [[-1.5325, -0.6184, -0.0688],
│                    [ 0.6734, -0.1891,  1.4454]],
│           
│                   [[-1.5325, -0.6184, -0.0688],
│                    [ 0.6734, -0.1891,  1.4454]]])
└── 'x' --> <FastTreeValue 0x7fa7d4dd8910>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.9 µs ± 38.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fa74a1d9700>
├── 'a' --> tensor([[-1.5325, -0.6184, -0.0688],
│                   [ 0.6734, -0.1891,  1.4454],
│                   [-1.5325, -0.6184, -0.0688],
│                   [ 0.6734, -0.1891,  1.4454],
│                   [-1.5325, -0.6184, -0.0688],
│                   [ 0.6734, -0.1891,  1.4454],
│                   [-1.5325, -0.6184, -0.0688],
│                   [ 0.6734, -0.1891,  1.4454],
│                   [-1.5325, -0.6184, -0.0688],
│                   [ 0.6734, -0.1891,  1.4454],
│                   [-1.5325, -0.6184, -0.0688],
│                   [ 0.6734, -0.1891,  1.4454],
│                   [-1.5325, -0.6184, -0.0688],
│                   [ 0.6734, -0.1891,  1.4454],
│                   [-1.5325, -0.6184, -0.0688],
│                   [ 0.6734, -0.1891,  1.4454]])
└── 'x' --> <FastTreeValue 0x7fa74a1d96d0>
    └── 'c' --> tensor([[-0.9273, -0.2462, -0.0223,  0.5596],
                        [-2.1601, -1.2822, -1.1878, -1.8289],
                 

In [23]:
%timeit t_cat(trees)

30.1 µs ± 33 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

55.7 µs ± 142 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-1.5325, -0.6184, -0.0688],
                [ 0.6734, -0.1891,  1.4454]],
       
               [[-1.5325, -0.6184, -0.0688],
                [ 0.6734, -0.1891,  1.4454]],
       
               [[-1.5325, -0.6184, -0.0688],
                [ 0.6734, -0.1891,  1.4454]],
       
               [[-1.5325, -0.6184, -0.0688],
                [ 0.6734, -0.1891,  1.4454]],
       
               [[-1.5325, -0.6184, -0.0688],
                [ 0.6734, -0.1891,  1.4454]],
       
               [[-1.5325, -0.6184, -0.0688],
                [ 0.6734, -0.1891,  1.4454]],
       
               [[-1.5325, -0.6184, -0.0688],
                [ 0.6734, -0.1891,  1.4454]],
       
               [[-1.5325, -0.6184, -0.0688],
                [ 0.6734, -0.1891,  1.4454]]]),
    x: Batch(
           c: tensor([[[-0.9273, -0.2462, -0.0223,  0.5596],
                       [-2.1601, -1.2822, -1.1878, -1.8289],
                       [-0.3527,  0.2015,  0.6889,  0.7537]],
         

In [26]:
%timeit Batch.stack(batches)

77.6 µs ± 284 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-1.5325, -0.6184, -0.0688],
               [ 0.6734, -0.1891,  1.4454],
               [-1.5325, -0.6184, -0.0688],
               [ 0.6734, -0.1891,  1.4454],
               [-1.5325, -0.6184, -0.0688],
               [ 0.6734, -0.1891,  1.4454],
               [-1.5325, -0.6184, -0.0688],
               [ 0.6734, -0.1891,  1.4454],
               [-1.5325, -0.6184, -0.0688],
               [ 0.6734, -0.1891,  1.4454],
               [-1.5325, -0.6184, -0.0688],
               [ 0.6734, -0.1891,  1.4454],
               [-1.5325, -0.6184, -0.0688],
               [ 0.6734, -0.1891,  1.4454],
               [-1.5325, -0.6184, -0.0688],
               [ 0.6734, -0.1891,  1.4454]]),
    x: Batch(
           c: tensor([[-0.9273, -0.2462, -0.0223,  0.5596],
                      [-2.1601, -1.2822, -1.1878, -1.8289],
                      [-0.3527,  0.2015,  0.6889,  0.7537],
                      [-0.9273, -0.2462, -0.0223,  0.5596],
                      [-2.1601, -

In [28]:
%timeit Batch.cat(batches)

145 µs ± 332 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

321 µs ± 1.98 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
